## 021.1. DEMO huggingface 使用共用 library folder 之 image容器 kernel
<span style="color:red">Change to Default kernel:   Image_G_pytorch_2.1.0-cuda11.8-cudnn8-devel</span>.

Model: https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GPTQ

ATLAS: https://atlas.nomic.ai/map/c1b88b47-2d9b-47e0-9002-b80766792582/2560fd25-52fe-42f1-a58f-ff5eccc890d2

### TRY IT
1. Lauch  a new notebook tab
2. Change to kernel:  Image_G_pytorch_2.1.0-cuda11.8-cudnn8-devel
3. In first cell add content below

```
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
```

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
!pip3 install huggingface-hub hf_transfer -q
!pip3 install transformers optimum -q
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7

In [ ]:
%%bash
# How to download, including from branches
mkdir -p Mistral-7B-OpenOrca-GPTQ
HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Mistral-7B-OpenOrca-GPTQ --local-dir Mistral-7B-OpenOrca-GPTQ --local-dir-use-symlinks False

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "./Mistral-7B-OpenOrca-GPTQ"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [ ]:
system_message="You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
prompt = "Tell me about AI"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])